# Métricas de concordância

Fontes importantes:
- Doc CSV: https://docs.python.org/3/library/csv.html
- Doc XML Element Tree: https://docs.python.org/3/library/xml.etree.elementtree.html

Esse script serve para salvar anotações em um arquivo CSV. Seu intuito é gerar uma representação de dados adequada à implementação de métricas de concordância.

## Geração do CSV a partir de XMLs

### Append de arquivos xml e instanciação de lista de roots

In [ ]:
import os

xmls = os.listdir('./xmls')

In [ ]:
xmls

['17_133.28.6.2013_v1.xml',
 '11_243.20.11.2013_v1.xml',
 '19_152.28.7.2014_v1.xml',
 '20_147.21.7.2014_v1.xml',
 '16_167.14.8.2013_v1.xml',
 '8_200.26.9.2013_v1.xml',
 '10_193.17.9.2013_v1.xml',
 '6_191.13.9.2013_v1.xml',
 '12_244.21.11.2013_v1.xml',
 '4_2.3.1.2013_v1.xml',
 '5_56.19.3.2013_v1.xml',
 '15_166.13.8.2013_v1.xml',
 '7_188.10.9.2013_v1.xml',
 '14_204.1.10.2013_v1.xml',
 '18_119.12.6.2013_v1.xml',
 '9_202.27.9.2013_v1.xml',
 '13_212.10.10.2013_v1.xml']

In [ ]:
import xml.etree.ElementTree as ET

# lista de roots para iteração posterior
roots = []

# populando lista de roots e iterando na lista xmls
for xml in xmls:
    xml = './xmls/' + xml
    tree = ET.parse(xml)
    root = tree.getroot()
    roots.append(root)

### Definição das colunas do CSV

O CSV foi montado com informações provientes do NidoTat. A ideia foi reunir todos os campos que podem vir a ser úteis na alimentação de métricas de concordância.

As colunas selecionadas foram:
- Id do DODF
- Tipo da relação
- Id da relação
- Anotador da relação
- Tipo da entidade
- Id da entidade
- Anotador da entidade
- Offset da entidade
- Length da entidade
- Texto da entidade

In [ ]:
import csv

# abre csv para escrita de headers
with open('agreement.csv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(['id_dodf', 'tipo_rel', 'id_rel', 'anotador_rel', 'tipo_ent', 'id_ent', 'anotador_ent', 'offset', 'length', 'texto'])

### XML -> CSV

In [ ]:
from tqdm.notebook import tqdm

# abre csv para escrita em modo append
with open('agreement.csv', 'a') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    
    # itera na lista de roots
    for root in tqdm(roots):
        # coleta id do dodf
        id_dodf = root.find("./document/id")
        id_dodf_text = id_dodf.text
        # cria lista de ids de relações
        ids_rel = []
        for rel in root.findall("./document/passage/relation"):
            id_rel = rel.get('id')
            ids_rel.append(id_rel)
            # coleta tipo e anotador da relação
            for info in rel.findall('infon'):
                if info.get('key') == 'type':
                    tipoRel = info.text
                    #print(tipoRel)
                elif info.get('key') == 'annotator':
                    annotatorRel = info.text
                    #print(annotatorRel)
            # cria lista de ids de anotações
            ids_anno = []
            for info in rel.findall('node'):
                id_anno = info.get('refid')
                ids_anno.append(id_anno)
            #print(ids_anno)
            # loop na lista de ids
            for id_anno in ids_anno:
                # encontra e itera sobre todos os elementos annotation do xml
                for anno in root.findall("./document/passage/annotation"):
                    # para cada anotação definida por um id, coleta o tipo, anotador, offset, length e texto
                    if anno.get('id') == id_anno:
                        # encontra tipo e anotador
                        for info in anno.findall('infon'):
                            if info.get('key') == 'type':
                                tipoAnno = info.text
                                #print(tipoAnno)
                            elif info.get('key') == 'annotator':
                                annotatorAnno = info.text
                                #print(annotatorAnno)
                        # encontra offset e length
                        for info in anno.findall('location'):
                            offset = info.get('offset')
                            #print(offset)
                            length = info.get('length')
                            #print(length)
                        # encontra texto
                        for info in anno.findall('text'):
                            texto = info.text
                            #print(texto)
                        # escreve linha no csv
                        writer.writerow([id_dodf_text, tipoRel, id_rel, annotatorRel, tipoAnno, id_anno, annotatorAnno, offset, length, texto])

  0%|          | 0/17 [00:00<?, ?it/s]

In [ ]:
import pandas as pd

df = pd.read_csv('agreement.csv')
df

,id_dodf,tipo_rel,id_rel,anotador_rel,tipo_ent,id_ent,anotador_ent,offset,length,texto
0,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,Ato_Exoneracao_Comissionado,1178,pedro_henrique,205008,326,EXONERAR SEBASTIAO FRANCISCO DE QUEIROZ do Car...
1,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,nome,1179,DODFMiner,205016,31,SEBASTIAO FRANCISCO DE QUEIROZ
2,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,simbolo,1181,DODFMiner,205078,4,DFA-
3,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,simbolo,1180,pedro_henrique,205078,7,DFA-\n12
4,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,cargo_comissionado,1182,DODFMiner,205089,9,Assessor
...,...,...,...,...,...,...,...,...,...,...
19487,13_212.10.10.2013,Ato_Substituicao,R40,khalil_carsten,matricula_substituido,8685,vinicius_borges,644324,9,174.833-5
19488,13_212.10.10.2013,Ato_Substituicao,R40,khalil_carsten,data_inicial,8686,vinicius_borges,644348,2,09
19489,13_212.10.10.2013,Ato_Substituicao,R40,khalil_carsten,data_final,8687,vinicius_borges,644353,10,11/10/2013
19490,13_212.10.10.2013,Ato_Substituicao,R40,khalil_carsten,motivo,8688,vinicius_borges,644380,32,ferias regulamentares da titular


In [ ]:
df.tipo_rel.value_counts()

Ato_Exoneracao_Comissionado       3891
Ato_Nomeacao_Comissionado         3851
Ato_Tornado_Sem_Efeito_Exo_Nom    2815
Ato_Retificacao_Comissionado      2691
Ato_Retificacao_Efetivo           1895
Ato_Substituicao                  1793
Ato_Exoneracao_Efetivo            1555
Ato_Cessao                         370
Ato_Nomeacao_Efetivo               197
Ato_Abono_Permanencia              175
Ato_Tornado_Sem_Efeito_Apo         170
Ato_Reversao                        89
Name: tipo_rel, dtype: int64

In [ ]:
df.tipo_ent.value_counts()

nome                          2022
orgao                         1600
hierarquia_lotacao            1512
cargo_comissionado            1411
simbolo                       1358
                              ... 
matricula_siape                  3
data_dodf_edital_normativo       3
data                             3
tipo_edicao                      2
especialidade                    1
Name: tipo_ent, Length: 69, dtype: int64

In [ ]:
df.anotador_rel.value_counts()

jose_reinaldo       3851
renato_nobre        3667
livia_fonseca       2564
matheus_stauffer    2413
khalil_carsten      2189
leonardo_maffei     1549
tatiana_franco      1443
pedro_henrique      1432
thiago_faleiros      384
Name: anotador_rel, dtype: int64

In [ ]:
df.anotador_ent.value_counts()

DODFMiner           4798
lindeberg           2878
tatiana_franco      2428
matheus_stauffer    2083
vinicius_borges     2057
patricia_medyna     1219
livia_fonseca       1118
pedro_henrique      1066
teo_decampos         753
leonardo_maffei      716
alvesisaque          141
jose_reinaldo        129
fredguth              60
gerente_matheus       28
khalil_carsten        13
thiago_faleiros        4
renato_nobre           1
Name: anotador_ent, dtype: int64

## Criação de estruturas auxiliares em memória

In [2]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

df = pd.read_csv('agreement.csv')
df

,id_dodf,tipo_rel,id_rel,anotador_rel,tipo_ent,id_ent,anotador_ent,offset,length,texto
0,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,Ato_Exoneracao_Comissionado,1178,pedro_henrique,205008,326,EXONERAR SEBASTIAO FRANCISCO DE QUEIROZ do Car...
1,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,nome,1179,DODFMiner,205016,31,SEBASTIAO FRANCISCO DE QUEIROZ
2,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,simbolo,1181,DODFMiner,205078,4,DFA-
3,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,simbolo,1180,pedro_henrique,205078,7,DFA-\n12
4,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,cargo_comissionado,1182,DODFMiner,205089,9,Assessor
...,...,...,...,...,...,...,...,...,...,...
19487,13_212.10.10.2013,Ato_Substituicao,R40,khalil_carsten,matricula_substituido,8685,vinicius_borges,644324,9,174.833-5
19488,13_212.10.10.2013,Ato_Substituicao,R40,khalil_carsten,data_inicial,8686,vinicius_borges,644348,2,09
19489,13_212.10.10.2013,Ato_Substituicao,R40,khalil_carsten,data_final,8687,vinicius_borges,644353,10,11/10/2013
19490,13_212.10.10.2013,Ato_Substituicao,R40,khalil_carsten,motivo,8688,vinicius_borges,644380,32,ferias regulamentares da titular


Para correlação offset/length/user/textos/anotadores/tipo_entidade, parece mais fácil extrair apenas a listagem em ordem e depois lidar com duplicatas.

In [3]:
offsets = df.offset.to_list()
#offsets

In [4]:
lengths = df.length.to_list()
#lengths

In [5]:
texts = df.texto.to_list()
#texts

In [6]:
anotators = df.anotador_ent.to_list()
#anotators

In [7]:
tipo_ent = df.tipo_ent.to_list()
#tipo_ent

Também extraímos offsets sem duplicata

In [8]:
off_sem_duplicata = np.unique(df.offset.to_numpy())

In [9]:
off_sem_duplicata

array([   304,    305,    312, ..., 644348, 644353, 644380])

`map_duplicata` é um dicionário cujo objetivo é dispor em uma estrutura de dados todas as ocorrências de um mesmo offset. Para tanto, mapeamos os offsets e linhas do dataframe original. As chaves são todos os offsets únicos, e os valores são listas contendo as linhas onde aquele offset ocorre. Com esse mapa, poderemos iterar entre os valores e adotar as diferentes métricas que quisermos.

In [10]:
map_duplicata = {}
for i in off_sem_duplicata:
    map_duplicata[i] = []
for idx, offset in enumerate(offsets):
    map_duplicata[offset].append(idx)

In [11]:
map_duplicata

{304: [15160],
 305: [1494],
 312: [9320, 9443, 9566, 9677],
 314: [15161],
 315: [1500],
 323: [15162],
 338: [9321, 9444, 9567, 9678],
 356: [1495],
 359: [12575],
 363: [9322, 9445, 9568, 9679, 15163],
 364: [9323, 9446, 9569, 9680],
 369: [12576],
 378: [12577],
 407: [9324, 9447, 9570, 9681],
 414: [15164],
 424: [15165],
 430: [1496],
 435: [9325, 9448, 9571, 9682],
 439: [12578],
 440: [1497],
 443: [15166],
 445: [9326, 9449, 9572, 9683],
 446: [9327, 9450, 9573, 9684],
 449: [12579],
 451: [1498],
 465: [12580],
 466: [1499],
 468: [9328, 9451, 9574, 9685],
 502: [1446],
 508: [1447],
 522: [9329, 9452, 9575, 9686],
 556: [9389, 9512, 9746, 9776],
 566: [1448],
 577: [9390, 9513, 9747, 9777, 15167],
 588: [9391, 9514, 9748, 9778],
 594: [1449],
 612: [15152],
 622: [15153],
 632: [9392, 9515, 9749, 9779],
 641: [9393, 9516, 9750, 9780],
 663: [15154],
 664: [15155],
 671: [9394, 9517, 9751, 9781],
 681: [1450],
 692: [9395, 9518, 9752, 9782],
 694: [12581],
 703: [15156],
 715

In [12]:
len(map_duplicata)

15566

In [13]:
map_duplicata[312]

[9320, 9443, 9566, 9677]

Para facilitar a consulta aos dados, vamos criar um dicionário `data_dict`, cujas chaves são linhas do dataframe e os valores são tuplas contendo `(offset, length, anotador_ent, texto)` da respectiva linha.

In [14]:
data_dict = {}
for idx, _ in enumerate(offsets):
    data_dict[idx] = (offsets[idx], lengths[idx], anotators[idx], tipo_ent[idx], texts[idx])

In [15]:
data_dict

{0: (205008,
  326,
  'pedro_henrique',
  'Ato_Exoneracao_Comissionado',
  'EXONERAR SEBASTIAO FRANCISCO DE QUEIROZ do Cargo em Comissao, Simbolo DFA-\n12, de Assessor, da Gerencia Regional do Setor Complementar de Industria e Abastecimento, da \nAdministracao Regional do Setor Complementar de Industria e Abastecimento, da Coordenadoria \ndas Cidades, da Casa Civil, da Governadoria do Distrito Federal.'),
 1: (205016, 31, 'DODFMiner', 'nome', ' SEBASTIAO FRANCISCO DE QUEIROZ'),
 2: (205078, 4, 'DODFMiner', 'simbolo', 'DFA-'),
 3: (205078, 7, 'pedro_henrique', 'simbolo', 'DFA-\n12'),
 4: (205089, 9, 'DODFMiner', 'cargo_comissionado', ' Assessor'),
 5: (205103,
  193,
  'DODFMiner',
  'hierarquia_lotacao',
  'Gerencia Regional do Setor Complementar de Industria e Abastecimento, da \nAdministracao Regional do Setor Complementar de Industria e Abastecimento, da Coordenadoria \ndas Cidades, da Casa Civil'),
 6: (205103,
  178,
  'pedro_henrique',
  'hierarquia_lotacao',
  'Gerencia Regional

In [16]:
len(data_dict)

19492

### Summary: estruturas auxiliares
A ideia é iterar em `map_duplicata`, obtendo a cada iteração listas de linhas do dataframe, e então usar `data_dict` - cujas chaves são as linhas do dataframe - para realizar o processamento necessário.

### Tratando exceções em map_duplicata

O dicionário `map_duplicata` mapeia offsets para linhas do dataframe original onde esse offset ocorre. Essa forma de pensar **parte da premissa que nossos anotadores anotaram o início de cada anotação corretamente**, o que nem sempre pode acontencer.

Para resolver esse problema, vamos adotar uma estratégia de buscar nos offsets vizinhos aos indicados por `map_duplicata` possíveis outras anotações. Para começar, vamos pensar nos casos de exceção que podem acontecer:
- O anotador **deixou de anotador 1 caractere no início;**
- O anotador **incluiu 1 caractere no início;**
- O anotador **deixou de anotador mais de 1 caractere no início (um ou mais termos, por exemplo);**
- O anotador **incluiu mais de 1 caractere no início (um ou mais termos, por exemplo);**
- O anotador **errou rude e anotou um texto muito diferente do que a anotação pedia:**
    - A anotação feita **foi muito menor do que o necessário;**
    - A anotação feita **foi muito maior do que o necessário;**

In [17]:
# exemplo base: anotação correta
t1 = 'EXONERAR SEBASTIAO FRANCISCO DE QUEIROZ do Cargo em Comissao, Simbolo DFA-\n12, de Assessor, da Gerencia Regional do Setor Complementar de Industria e Abastecimento, da \nAdministracao Regional do Setor Complementar de Industria e Abastecimento, da Coordenadoria \ndas Cidades, da Casa Civil, da Governadoria do Distrito Federal.'
off1 = 205008
print(len(t1))
# engoliu um caractere no início
t2 = 'XONERAR SEBASTIAO FRANCISCO DE QUEIROZ do Cargo em Comissao, Simbolo DFA-\n12, de Assessor, da Gerencia Regional do Setor Complementar de Industria e Abastecimento, da \nAdministracao Regional do Setor Complementar de Industria e Abastecimento, da Coordenadoria \ndas Cidades, da Casa Civil, da Governadoria do Distrito Federal.'
off2 = 205009
print(len(t2))
# incluiu um caractere no inicio
t3 = ' EXONERAR SEBASTIAO FRANCISCO DE QUEIROZ do Cargo em Comissao, Simbolo DFA-\n12, de Assessor, da Gerencia Regional do Setor Complementar de Industria e Abastecimento, da \nAdministracao Regional do Setor Complementar de Industria e Abastecimento, da Coordenadoria \ndas Cidades, da Casa Civil, da Governadoria do Distrito Federal.'
off3 = 205007
print(len(t3))
# engoliu mais de um caractere (um ou + termos, por exemplo) no início
t4 = 'SEBASTIAO FRANCISCO DE QUEIROZ do Cargo em Comissao, Simbolo DFA-\n12, de Assessor, da Gerencia Regional do Setor Complementar de Industria e Abastecimento, da \nAdministracao Regional do Setor Complementar de Industria e Abastecimento, da Coordenadoria \ndas Cidades, da Casa Civil, da Governadoria do Distrito Federal.'
off4 = 205217
print(len(t4))
# incluiu mais de um caractere (um ou + termos, por exemplo) no início
t5 = 'do Distrito Federal. EXONERAR SEBASTIAO FRANCISCO DE QUEIROZ do Cargo em Comissao, Simbolo DFA-\n12, de Assessor, da Gerencia Regional do Setor Complementar de Industria e Abastecimento, da \nAdministracao Regional do Setor Complementar de Industria e Abastecimento, da Coordenadoria \ndas Cidades, da Casa Civil, da Governadoria do Distrito Federal.'
off5 = 204986
print(len(t5))
# anotação muito pequena para passar no teste de tamanho
t6 = 'não é para entrar'
off6 = 200000
print(len(t6))
# anotação muito grande para passar no teste de tamanho
t7 = 'do Distrito Federal. EXONERAR SEBASTIAO FRANCISCO DE QUEIROZ do Cargo em Comissao, Simbolo DFA-\n12, de Assessor, da Gerencia Regional do Setor Complementar de Industria e Abastecimento, da \nAdministracao Regional do Setor Complementar de Industria e Abastecimento, da Coordenadoria \ndas Cidades, da Casa Civil, da Governadoria do Distrito Federal.do Distrito Federal. EXONERAR SEBASTIAO FRANCISCO DE QUEIROZ do Cargo em Comissao, Simbolo DFA-\n12, de Assessor, da Gerencia Regional do Setor Complementar de Industria e Abastecimento, da \nAdministracao Regional do Setor Complementar de Industria e Abastecimento, da Coordenadoria \ndas Cidades, da Casa Civil, da Governadoria do Distrito Federal.do Distrito Federal. EXONERAR SEBASTIAO FRANCISCO DE QUEIROZ do Cargo em Comissao, Simbolo DFA-\n12, de Assessor, da Gerencia Regional do Setor Complementar de Industria e Abastecimento, da \nAdministracao Regional do Setor Complementar de Industria e Abastecimento, da Coordenadoria \ndas Cidades, da Casa Civil, da Governadoria do Distrito Federal.do Distrito Federal. EXONERAR SEBASTIAO FRANCISCO DE QUEIROZ do Cargo em Comissao, Simbolo DFA-\n12, de Assessor, da Gerencia Regional do Setor Complementar de Industria e Abastecimento, da \nAdministracao Regional do Setor Complementar de Industria e Abastecimento, da Coordenadoria \ndas Cidades, da Casa Civil, da Governadoria do Distrito Federal.'
off7 = 300000
print(len(t7))

326
325
327
317
347
17
1388


Ótimo, temos alguns exemplos para testar. Bom, desses listados, quem entra? O primeiro critério seria baseado no até que posição do DODF devemos verificar. Nesse sentido, vamos olhar para offsets vizinhos ao original - lembrando que **offset é o ponteiro para o 1º caractere da anotação**. O limiar seria definido por uma varíavel **threshold**, um número inteiro que representa a quantidade de caracteres que vamos andar a partir do offset mapeado originalmente. Por exemplo, se tivermos `threshold = 5` e `offset_referencia = 200500`, vamos verificar se existem anotações nos offsets `[200495, 200496, 200497, 200498, 200499]` (os 5 para trás do original) e `[200501, 200502, 200503, 200504, 200505]` (os 5 para frente do original). Caso existam, podemos prosseguir para o próximo teste.

Esse threshold será definido pelo inteiro mais próximo de 10% do tamanho do texto da anotação original em caracteres.

In [18]:
import math

In [19]:
threshold_init = math.ceil(len(t1)/10)
threshold = threshold_init if threshold_init > 1 else 1
threshold

33

Como o tamanho em caracteres de `t1` é 326, seu threshold será igual a 33.

Também verificaremos se o tipo de entidade da anotação obtida com threshold é o mesmo da anotação base. Como vamos andar alguns caracteres para frente e para trás, vamos encontrar muito possivelmente anotações, mas de tipos distintos.

Outro critério é, uma vez que os textos foram selecionados, verificar se a string referente à anotação original é substring da string obtida com o threshold de offsets ou vice-versa. Aqui precisaremos usar novamente a ideia de threshold para determinar limites aceitáveis para testar substrings. Por exemplo, se não adicionássemos esse teste, o caso de `a` ser substring de `EXONERAR SEBASTIAO FRANCISCO DE QUEIROZ do Cargo em Comissao, Simbolo DFA-\n12, de Assessor, da Gerencia Regional do Setor Complementar de Industria e Abastecimento, da \nAdministracao Regional do Setor Complementar de Industria e Abastecimento, da Coordenadoria \ndas Cidades, da Casa Civil, da Governadoria do Distrito Federal.` passaria, mas não é isso que queremos.

Novamente, vamos usar como referência uma diferença de 10% de tamanho em relação à string da anotação base. Vamos ver um exemplo desse último ponto aqui abaixo:

In [20]:
pool_offsets = []
toy_list_offsets = []
toy_list_textos = []

toy_list_offsets.append(off1)

pool_offsets.append(off2)
pool_offsets.append(off3)
pool_offsets.append(off4)
pool_offsets.append(off5)
pool_offsets.append(off6)
pool_offsets.append(off7)

toy_list_textos.append(t2)
toy_list_textos.append(t3)
toy_list_textos.append(t4)
toy_list_textos.append(t5)
toy_list_textos.append(t6)
toy_list_textos.append(t7)

for idx, item in enumerate(toy_list_textos):
    if(len(item) > len(t1)):
        # se o tamanho de item for no máximo tamanho 
        # anotação referência + 10% tamanho anotação referência
        if(len(item) < math.floor(len(t1)*(11/10))):
            if t1 in item:
                toy_list_offsets.append(pool_offsets[idx])
                print(f'entrei! t1 > item. append em {pool_offsets[idx]}')
    else:
        # se o tamanho dela for no mínimo 90% do tamanho da anotação referência
        if(len(item) > math.floor(len(t1)*(9/10))):
            if item in t1:
                toy_list_offsets.append(pool_offsets[idx])
                print(f'entrei! item > t1. append em {pool_offsets[idx]}')

entrei! item > t1. append em 205009
entrei! t1 > item. append em 205007
entrei! item > t1. append em 205217
entrei! t1 > item. append em 204986


In [21]:
toy_list_offsets

[205008, 205009, 205007, 205217, 204986]

Das strings que setamos antes, `t6` e `t7` não entram pela limitação de tamanho, assim como esperado.

Recapitulando, o tratamento de duplicatas será feito com três testes:

1. Verificamos offsets vizinhos ao da anotação referência. A quantidade de offsets verificados será dada por um threshold equivalente à 10% do tamanho da anotação referência;
2. Se houver anotação em algum offset vizinho, verificamos se o label dessa anotação é o mesmo da anotação referência;
3. Se houver anotação em um offset vizinho e essa anotação for do mesmo tipo que a anotação referência, verificamos o tamanho da anotação do offset vizinho. Novamente usamos a ideia de threshold aqui: a nova anotação deve ter um tamanho entre $90\% \text{ e } 110\%$ ao da anotação referência.

In [22]:
map_duplicata_final = map_duplicata

for off_ref in tqdm(map_duplicata):
    lista_linha_dataframe = map_duplicata[off_ref]
    # vamos andar pra trás threshold caracteres e depois pra frente threshold caracteres
    for linha_dataframe in lista_linha_dataframe:
        threshold_init = math.ceil(len(data_dict[linha_dataframe][4])/10)
        threshold = threshold_init if threshold_init > 1 else 1
        for i in range(threshold):
            # andando pra trás
            try:
                # linha mapeada por offset_referencia - threshold da vez
                linha_threshold = data_dict[linha_dataframe-(threshold-i)][0]
                
                # label da anotação referência
                label_ref = data_dict[linha_dataframe][3]
                
                # label da anotação referente à linha mapeada por offset_referencia - threshold da vez
                label_threshold = data_dict[linha_dataframe-(threshold-i)][3]
                
                # texto da anotação mapeada por linha dataframe
                anotacao_referencia = data_dict[linha_dataframe][4]
                
                # texto da anotação mapeada por linha threshold
                anotacao_threshold = data_dict[linha_threshold][4]

                # verifica tamanho das duas anotações para testar se é substring
                if(len(anotacao_threshold) > len(anotacao_referencia)):
                    # se existe anotação na linha_threshold
                    if(map_duplicata[linha_threshold]):
                        # se o label dela for igual ao label da anotação referência
                        if(label_ref == label_threshold):
                            # se o tamanho dela for no máximo tamanho 
                            # anotação referência + 10% tamanho anotação referência
                            if(len(anotacao_threshold) < math.floor(len(anotacao_referencia)*(11/10))):
                                if anotacao_referencia in anotacao_threshold:
                                    map_duplicata_final[linha_dataframe].append(linha_threshold)
                else:
                    # se existe anotação na linha_threshold
                    if(map_duplicata[linha_threshold]):
                        # se o label dela for igual ao label da anotação referência
                        if(label_ref == label_threshold):
                            # se o tamanho dela for no mínimo 90% do tamanho da anotação referência
                            if(len(anotacao_threshold) > math.floor(len(anotacao_referencia)*(9/10))):
                                if anotacao_threshold in anotacao_referencia:
                                    map_duplicata_final[linha_dataframe].append(linha_threshold)
            except:
                pass
            # andando pra frente
            try:
                # linha mapeada por offset_referencia - threshold da vez
                linha_threshold = data_dict[linha_dataframe+(i+1)][0]
                
                # label da anotação referência
                label_ref = data_dict[linha_dataframe][3]
                
                # label da anotação referente à linha mapeada por offset_referencia - threshold da vez
                label_threshold = data_dict[linha_dataframe+(i+1)][3]
                
                # texto da anotação mapeada por linha dataframe
                anotacao_referencia = data_dict[linha_dataframe][4]
                
                # texto da anotação mapeada por linha threshold
                anotacao_threshold = data_dict[linha_threshold][4]
                
                # verifica tamanho das duas anotações para testar se é substring
                if(len(anotacao_threshold) > len(anotacao_referencia)):
                    # se existe anotação na linha_threshold
                    if(map_duplicata[linha_threshold]):
                        # se o label dela for igual ao label da anotação referência
                        if(label_ref == label_threshold):
                            # se o tamanho dela for no máximo tamanho 
                            # anotação referência + 10% tamanho anotação referência
                            if(len(anotacao_threshold) < math.floor(len(anotacao_referencia)*(11/10))):
                                if anotacao_referencia in anotacao_threshold:
                                    map_duplicata_final[linha_dataframe].append(linha_threshold)
                else:
                    # se existe anotação na linha_threshold
                    if(map_duplicata[linha_threshold]):
                        # se o label dela for igual ao label da anotação referência
                        if(label_ref == label_threshold):
                            # se o tamanho dela for no mínimo 90% do tamanho da anotação referência
                            if(len(anotacao_threshold) > math.floor(len(anotacao_referencia)*(9/10))):
                                if anotacao_threshold in anotacao_referencia:
                                    map_duplicata_final[linha_dataframe].append(linha_threshold)
            except:
                pass

  0%|          | 0/15566 [00:00<?, ?it/s]

Nosso mapeamento final, dado o código acima, é dado por `map_duplicata_final`.

## Métricas de similaridade

### Jaccard

O coeficiente de Jaccard foi originalmente criado para imagens, mas podemos adaptar para dados textuais.

<img src="jaccard.png" alt="Drawing" style="width: 600px;"/>

<img src="jaccard2.png" alt="Drawing" style="width: 600px;"/>

Fonte das imagens: https://medium.com/data-science-bootcamp/understand-jaccard-index-jaccard-similarity-in-minutes-25a703fbf9d7

Na prática, nossa adaptação para textos vai assumir que a união entre as strings A e B é a string de maior tamanho. Como estamos comparando - em tese - strings parecidas, pensando a nível de caractere faz sentido pensar nessa união como um overlap de textos parecidos. Essa premissa já tras consigo um certo bias, mas acho que é uma assumption aceitável. Se a união é o overlap, a interseção é justamente esse overlap menos o que A e B têm de diferente. Basta encontrar a quantidade de caracteres que A e B diferem e tomar o módulo da diferença com a string de maior tamanho.

Em uma formulinha, ficaria assim:

$$\text{Jaccard}(A, B) = \frac{|{\text{tam_str} - \text{diferenca_char}}|}{\text{tam_str}}$$

onde _tam_str_ é o tamanho de uma das strings avaliadas - o módulo permite pegar qualquer uma - e _diferenca_char_ a diferença de caracteres entre A e B.

Um passo importante para computar a diferença de caracteres corretamente é levar em conta casos onde um das strings possa estar deslocada por um ou mais caracteres da outra string dada como argumento. Observe os textos abaixo:

In [23]:
t1, t4

('EXONERAR SEBASTIAO FRANCISCO DE QUEIROZ do Cargo em Comissao, Simbolo DFA-\n12, de Assessor, da Gerencia Regional do Setor Complementar de Industria e Abastecimento, da \nAdministracao Regional do Setor Complementar de Industria e Abastecimento, da Coordenadoria \ndas Cidades, da Casa Civil, da Governadoria do Distrito Federal.',
 'SEBASTIAO FRANCISCO DE QUEIROZ do Cargo em Comissao, Simbolo DFA-\n12, de Assessor, da Gerencia Regional do Setor Complementar de Industria e Abastecimento, da \nAdministracao Regional do Setor Complementar de Industria e Abastecimento, da Coordenadoria \ndas Cidades, da Casa Civil, da Governadoria do Distrito Federal.')

Se olharmos cegamente para cada caractere, **partindo do primeiro**, teremos um score de similaridade bem baixo. No entanto, as strings são bem parecidas; na realidade, `t4` é uma substring de `t1`, e está deslocada alguns caracteres.

Nesse cenário, precisamos computar se alguma das strings dadas como argumento é substring da outra. Se for, partimos do caractere onde elas começam a 'ser iguais'. Na prática, é como se fizéssemos um shift até igualar os textos.

Em termos de codificação, para fazer esse teste usamos a função `find`. Para `t1` e `t4`, temos que `t4` é substring de `t1`, e o primeiro caractere de overlap é o 9.

In [24]:
t1.find(t4)

9

Nossa função final fica assim:

In [321]:
def jaccard_char(str1, str2):
    diferenca_char = 0
    # string 2 maior que string 1
    if len(str2) > len(str1):
        inicio = str2.find(str1)
        # teste substring
        if inicio == -1:
            agreement = 0
            return agreement
        else:
            idx_menor = 0
            for idx, _ in enumerate(str2):
                try:
                    # shift na string até encontrar o caractere de inicio
                    if idx < inicio:
                        diferenca_char += 1
                        # contador para shift-reverso
                        idx_menor +=1
                    else:
                        # strings 'alinhadas', diferença char a char
                        if str2[idx] != str1[idx-idx_menor]:
                            diferenca_char += 1
                # acabamos de percorrer str1; computamos a diferença remanescente em relação à str2
                except:
                    dif_reman = len(str2) - idx
                    diferenca_char += dif_reman
                    break
            # computamos o agreement usando jaccard
            agreement = abs(len(str2) - diferenca_char)/len(str2)
            
    # string 1 maior que string 2
    elif len(str1) > len(str2):
        # teste substring
        inicio = str1.find(str2)
        if inicio == -1:
            agreement = 0
            return agreement
        else:
            idx_menor = 0
            for idx, _ in enumerate(str1):
                try:
                    # shift na string até encontrar o caractere de inicio
                    if idx < inicio:
                        diferenca_char += 1
                        # contador para shift-reverso
                        idx_menor +=1
                    else:
                        # strings 'alinhadas', diferença char a char
                        if str1[idx] != str2[idx-idx_menor]:
                            diferenca_char += 1
                # acabamos de percorrer str2; computamos a diferença remanescente em relação à str1
                except:
                    dif_reman = len(str1) - idx
                    diferenca_char += dif_reman
                    break
            # computamos o agreement usando jaccard
            agreement = abs(len(str1) - diferenca_char)/len(str1)
            
    # strings de mesmo tamanho
    else:
        # teste subtring
        inicio = str1.find(str2)
        if inicio == -1:
            agreement = 0
            return agreement
        # str1 e str2 são a mesma string
        elif inicio == 0:
            agreement = 1
        else:
            # strings alinhadas, diferença char a char
            for idx, _ in enumerate(str1):
                if str1[idx] != str2[idx]:
                    diferenca_char += 1
            total_dif = abs(len(str1) - diferenca_char)/len(str1)
            print(f'total_dif caso 3 = {total_dif}')
            agreement = 1 - total_dif
    
    return agreement

No exemplo com `t1` e `t4`, temos:

In [322]:
jaccard_char(t4, t1)

0.9723926380368099

Em outro exemplo de uso, vamos pegar essas duas anotações de símbolo:

In [323]:
data_dict[2][4], data_dict[3][4]

('DFA-', 'DFA-\n12')

In [324]:
agreement = jaccard_char(data_dict[2][4], data_dict[3][4])
agreement

0.5714285714285714

Ou, pensando em diferença:

In [325]:
dif = 1 - agreement
dif

0.4285714285714286

### Similaridade de cossenos

Outra abordagem possível para medir diferença entre textos é usar técnicas baseadas em representações vetoriais, como a similaridade de cossenos e distância euclidiana.

In [29]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

df = pd.read_csv('agreement.csv')
df

,id_dodf,tipo_rel,id_rel,anotador_rel,tipo_ent,id_ent,anotador_ent,offset,length,texto
0,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,Ato_Exoneracao_Comissionado,1178,pedro_henrique,205008,326,EXONERAR SEBASTIAO FRANCISCO DE QUEIROZ do Car...
1,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,nome,1179,DODFMiner,205016,31,SEBASTIAO FRANCISCO DE QUEIROZ
2,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,simbolo,1181,DODFMiner,205078,4,DFA-
3,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,simbolo,1180,pedro_henrique,205078,7,DFA-\n12
4,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,cargo_comissionado,1182,DODFMiner,205089,9,Assessor
...,...,...,...,...,...,...,...,...,...,...
19487,13_212.10.10.2013,Ato_Substituicao,R40,khalil_carsten,matricula_substituido,8685,vinicius_borges,644324,9,174.833-5
19488,13_212.10.10.2013,Ato_Substituicao,R40,khalil_carsten,data_inicial,8686,vinicius_borges,644348,2,09
19489,13_212.10.10.2013,Ato_Substituicao,R40,khalil_carsten,data_final,8687,vinicius_borges,644353,10,11/10/2013
19490,13_212.10.10.2013,Ato_Substituicao,R40,khalil_carsten,motivo,8688,vinicius_borges,644380,32,ferias regulamentares da titular


A ideia aqui é criar uma representação vetorial de nossos dados a partir do algoritmo word2vec.

Inicialmente, vamos criar duas listas: uma contendo todos os textos de anotações e outra contendo todos os termos, vulgo tokens, de cada texto de anotação. Essa última será uma lista de listas.

In [30]:
anotacoes = df['texto'].to_list()

In [31]:
len(anotacoes)

19492

In [32]:
anotacoes[0]

'EXONERAR SEBASTIAO FRANCISCO DE QUEIROZ do Cargo em Comissao, Simbolo DFA-\n12, de Assessor, da Gerencia Regional do Setor Complementar de Industria e Abastecimento, da \nAdministracao Regional do Setor Complementar de Industria e Abastecimento, da Coordenadoria \ndas Cidades, da Casa Civil, da Governadoria do Distrito Federal.'

Para a lista de tokens, vamos desprezar qualquer pré-processamento. Queremos obter uma representação dos dados o mais próximo possível da realidade, então usaremos apenas o método `split()` para separar os tokens de cada texto de anotação.

In [33]:
anotacoes[0].split()

['EXONERAR',
 'SEBASTIAO',
 'FRANCISCO',
 'DE',
 'QUEIROZ',
 'do',
 'Cargo',
 'em',
 'Comissao,',
 'Simbolo',
 'DFA-',
 '12,',
 'de',
 'Assessor,',
 'da',
 'Gerencia',
 'Regional',
 'do',
 'Setor',
 'Complementar',
 'de',
 'Industria',
 'e',
 'Abastecimento,',
 'da',
 'Administracao',
 'Regional',
 'do',
 'Setor',
 'Complementar',
 'de',
 'Industria',
 'e',
 'Abastecimento,',
 'da',
 'Coordenadoria',
 'das',
 'Cidades,',
 'da',
 'Casa',
 'Civil,',
 'da',
 'Governadoria',
 'do',
 'Distrito',
 'Federal.']

In [34]:
tokens = []

for texto in tqdm(anotacoes):
    token_list = texto.split()
    tokens.append(token_list)

  0%|          | 0/19492 [00:00<?, ?it/s]

In [35]:
len(tokens)

19492

Com a lista de tokens em mãos, o que falta agora é criar uma representação vetorial em si. Vamos usar a suíte de word2vec do gensim, que poupa muito trabalho. Basta passar como parâmetro a lista de tokens e a dimensão das embeddings - aqui vamos de 50. Os outros parâmetros são optativos; aqui usamos:
- window, que diz respeito à janela de contexto usada na geração dos vetores. Imagine o seguinte texto: 'embeddings são interessantes e úteis'. Se o valor escolhido é 3, o vetor gerado referente ao termos 'embeddings' será 'influenciado' pelos 3 termos seguintes, 'são', 'interessantes' e 'e';
- min_count, serve para ignorar os termos com frequência menor que o inteiro informado;
- workers, quantas worker threads serão usadas para treinar o modelo;
- negative, que diz o número de noise words para serem usadas no processo de negative sampling. Se o número setado for 0, negative sampling não é usado.

Mais detalhes constam em https://radimrehurek.com/gensim/models/word2vec.html.

In [36]:
from gensim.models import Word2Vec

In [37]:
model = Word2Vec(sentences=tokens, size=50, window=3, min_count=1, workers=4, negative=5)

In [38]:
vectors = model.wv

Temos vetores de como os tokens vieram nas anotações. Apenas para reforçar, usamos apenas split na criação da lista de termos propositalmente, para conseguir representar a realidade dos dados da maneira mais fiel possível. Temos vetores para strings como '11/10/2013', por exemplo. Esse caso que pode ocorrer em entidades como vigência ou data.

In [39]:
vectors['11/10/2013']

array([ 0.04065349,  0.02935317, -0.01174439,  0.03280368,  0.02004511,
        0.03065644, -0.03564082, -0.00910922, -0.05266728,  0.02409262,
       -0.05473151,  0.02183036, -0.00649032,  0.02279046, -0.03407939,
       -0.00751406,  0.01435575, -0.02976743, -0.04140944, -0.02745342,
        0.03597784,  0.00137919, -0.08724046,  0.00934316, -0.02494546,
        0.02808123, -0.04222945,  0.0239162 ,  0.00679384, -0.02349762,
       -0.00371245, -0.00151136, -0.00762578, -0.060404  ,  0.00421406,
        0.01060077,  0.03223737,  0.01960725, -0.01442477, -0.0084267 ,
        0.0092442 ,  0.00482579, -0.01363936, -0.02310625, -0.04287918,
       -0.02625135, -0.00381688,  0.01776855,  0.02267743, -0.02497202],
      dtype=float32)

Criamos o modelo vetorial termo a termo. Agora, vamos obter vetores para os textos de anotação - que podem ter mais de um único token - somando a representação vetorial de cada termo no texto, de maneira a retornar um único vetor de 50 dimensões.

Para tanto, vamos criar uma função para facilitar nossa vida. A função `get_vector` recebe um texto como parâmetro e retorna uma representação vetorial correspondente.

In [40]:
def get_vector(text):
    token_list = text.split()
    vector = np.zeros(50)
    for token in token_list:
        vector += vectors[token]
    return vector

Para a primeira anotação do dataframe, temos o seguinte:

In [41]:
anotacoes[0]

'EXONERAR SEBASTIAO FRANCISCO DE QUEIROZ do Cargo em Comissao, Simbolo DFA-\n12, de Assessor, da Gerencia Regional do Setor Complementar de Industria e Abastecimento, da \nAdministracao Regional do Setor Complementar de Industria e Abastecimento, da Coordenadoria \ndas Cidades, da Casa Civil, da Governadoria do Distrito Federal.'

In [42]:
get_vector(anotacoes[0])

array([ 44.37932   ,  15.80390213, -40.26033535,  27.65815832,
       -12.64881931,  42.10618606, -13.66780284, -44.47665798,
       -33.7967721 ,  -4.07344657, -61.60341278,  16.3947466 ,
        26.23677571,  48.81898403, -93.31881671, -11.47885536,
        59.77060722, -38.47097494, -10.79075962,   1.3814194 ,
        15.38925025, -32.88735554, -42.55821792,  -1.61958386,
       -12.44505426,  18.58515903, -79.25654734,  41.39137798,
        10.99320741,  -2.81077432, -12.28320238,  17.10757984,
       -12.0292094 , -84.5813254 , -12.7074681 ,  -3.22436536,
        45.66052075, -52.0144837 ,  35.02840738, -61.55918618,
         1.00682802,  40.78993418,   1.4223363 , -21.1592781 ,
       -40.09794957,  25.30896906,  20.17197137,  47.71125761,
       -56.27870592,  24.1976793 ])

Para auxiliar no mapeamento da referência da anotação de texto (linha do dataframe) e sua representação vetorial, vamos criar uma nova coluna index. A ideia é bem simples: criar uma lista de 19492 inteiros e agregar ao dataframe.

In [43]:
index = [x for x in range (19492)]

In [44]:
len(index)

19492

In [45]:
df['index'] = index

In [46]:
df

,id_dodf,tipo_rel,id_rel,anotador_rel,tipo_ent,id_ent,anotador_ent,offset,length,texto,index
0,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,Ato_Exoneracao_Comissionado,1178,pedro_henrique,205008,326,EXONERAR SEBASTIAO FRANCISCO DE QUEIROZ do Car...,0
1,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,nome,1179,DODFMiner,205016,31,SEBASTIAO FRANCISCO DE QUEIROZ,1
2,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,simbolo,1181,DODFMiner,205078,4,DFA-,2
3,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,simbolo,1180,pedro_henrique,205078,7,DFA-\n12,3
4,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,cargo_comissionado,1182,DODFMiner,205089,9,Assessor,4
...,...,...,...,...,...,...,...,...,...,...,...
19487,13_212.10.10.2013,Ato_Substituicao,R40,khalil_carsten,matricula_substituido,8685,vinicius_borges,644324,9,174.833-5,19487
19488,13_212.10.10.2013,Ato_Substituicao,R40,khalil_carsten,data_inicial,8686,vinicius_borges,644348,2,09,19488
19489,13_212.10.10.2013,Ato_Substituicao,R40,khalil_carsten,data_final,8687,vinicius_borges,644353,10,11/10/2013,19489
19490,13_212.10.10.2013,Ato_Substituicao,R40,khalil_carsten,motivo,8688,vinicius_borges,644380,32,ferias regulamentares da titular,19490


Já temos os elementos para processar em massa! Vamos criar um estrutura para mapear anotações e representações vetorias. Nesse sentido, temos o dicionário `map_texto_vetores`, quase autoexplicativo: mapeia textos e seus respectivos vetores. As listas `index` e `anotacoes` tem o mesmo tamanho. Iterando em `index`, teremos as mesmas referências para a lista `anotacoes`. Usamos a função `create_vector` feita mais cedo para retornar cada vetor.

_disclaimer_: Podia ter feito só um enumerate na lista `anotacoes` que aqui teria o mesmo resultado. De todo modo, criar uma coluna de index no dataframe pode ser útil em outros momentos do trabalho.

In [47]:
map_texto_vetores = {}

for i in tqdm(index):
    map_texto_vetores[i] = get_vector(anotacoes[i])

  0%|          | 0/19492 [00:00<?, ?it/s]

In [48]:
map_texto_vetores

{0: array([ 44.37932   ,  15.80390213, -40.26033535,  27.65815832,
        -12.64881931,  42.10618606, -13.66780284, -44.47665798,
        -33.7967721 ,  -4.07344657, -61.60341278,  16.3947466 ,
         26.23677571,  48.81898403, -93.31881671, -11.47885536,
         59.77060722, -38.47097494, -10.79075962,   1.3814194 ,
         15.38925025, -32.88735554, -42.55821792,  -1.61958386,
        -12.44505426,  18.58515903, -79.25654734,  41.39137798,
         10.99320741,  -2.81077432, -12.28320238,  17.10757984,
        -12.0292094 , -84.5813254 , -12.7074681 ,  -3.22436536,
         45.66052075, -52.0144837 ,  35.02840738, -61.55918618,
          1.00682802,  40.78993418,   1.4223363 , -21.1592781 ,
        -40.09794957,  25.30896906,  20.17197137,  47.71125761,
        -56.27870592,  24.1976793 ]),
 1: array([ 4.63882475, -1.68105008,  0.85466271,  1.67372237,  3.94574966,
         1.57651293, -1.74165531, -3.14090145, -2.55490774,  0.78363122,
        -3.33631749,  1.00196544, -4.08057

Mais cedo citamos similaridade de cossenos e distância euclidiana. No curso do Coursera, tivemos de fazer as duas implementações, que são bem simples. Segue na íntegra os enunciados e resultados.

The cosine similarity function is:

$$\cos (\theta)=\frac{\mathbf{A} \cdot \mathbf{B}}{\|\mathbf{A}\|\|\mathbf{B}\|}=\frac{\sum_{i=1}^{n} A_{i} B_{i}}{\sqrt{\sum_{i=1}^{n} A_{i}^{2}} \sqrt{\sum_{i=1}^{n} B_{i}^{2}}}\tag{1}$$

$A$ and $B$ represent the word vectors and $A_i$ or $B_i$ represent index i of that vector.
& Note that if A and B are identical, you will get $cos(\theta) = 1$.
* Otherwise, if they are the total opposite, meaning, $A= -B$, then you would get $cos(\theta) = -1$.
* If you get $cos(\theta) =0$, that means that they are orthogonal (or perpendicular).
* Numbers between 0 and 1 indicate a similarity score.
* Numbers between -1-0 indicate a dissimilarity score.

**Instructions**: Implement a function that takes in two word vectors and computes the cosine distance.

In [49]:
def cos_sim(A, B):
    dot = np.dot(A, B)
    norma = np.linalg.norm(A)
    normb = np.linalg.norm(B)
    cos = dot/(norma * normb)
    
    return cos

You will now implement a function that computes the similarity between two vectors using the Euclidean distance.
Euclidean distance is defined as:

$$ \begin{aligned} d(\mathbf{A}, \mathbf{B})=d(\mathbf{B}, \mathbf{A}) &=\sqrt{\left(A_{1}-B_{1}\right)^{2}+\left(A_{2}-B_{2}\right)^{2}+\cdots+\left(A_{n}-B_{n}\right)^{2}} \\ &=\sqrt{\sum_{i=1}^{n}\left(A_{i}-B_{i}\right)^{2}} \end{aligned}$$

* $n$ is the number of elements in the vector
* $A$ and $B$ are the corresponding word vectors. 
* The more similar the words, the more likely the Euclidean distance will be close to 0. 

**Instructions**: Write a function that computes the Euclidean distance between two vectors.

In [50]:
def euclidean(A, B):
    d = np.linalg.norm(A-B)
    return d

Usando os mesmos exemplos que fizemos com a medida de Jaccard, temos o seguinte:

In [51]:
data_dict[2][4], data_dict[3][4]

('DFA-', 'DFA-\n12')

In [52]:
cos_sim(get_vector(data_dict[2][4]), get_vector(data_dict[3][4]))

0.9189129167781304

In [53]:
euclidean(get_vector(data_dict[2][4]), get_vector(data_dict[3][4]))

1.971498403251253

### Cohen Kappa

Referências interessantes:
- https://towardsdatascience.com/assessing-annotator-disagreements-in-python-to-build-a-robust-dataset-for-machine-learning-16c74b49f043
- https://github.com/o-P-o/disagree
- https://pypi.org/project/krippendorff/
- https://www.youtube.com/watch?v=fOR_8gkU3UE

$$K = \frac{\text{Concordância observada} - \text{Concordância por chance}}{1 - \text{Concordância por chance}}$$

co = diagonal principal de uma matriz de confusão, ambos marcaram igual e com os mesmos labels
ca = ambos marcaram os mesmos labels, não necessariamente iguais

In [ ]:
map_duplicata[205008]

[0, 9, 18, 35]

In [ ]:
for linha in map_duplicata_final[205008]:
    print(data_dict[linha][3])

Ato_Exoneracao_Comissionado
Ato_Exoneracao_Comissionado
Ato_Exoneracao_Comissionado
Ato_Exoneracao_Comissionado


In [ ]:
tipo_ent

['Ato_Exoneracao_Comissionado',
 'nome',
 'simbolo',
 'simbolo',
 'cargo_comissionado',
 'hierarquia_lotacao',
 'hierarquia_lotacao',
 'orgao',
 'orgao',
 'Ato_Exoneracao_Comissionado',
 'nome',
 'simbolo',
 'simbolo',
 'cargo_comissionado',
 'hierarquia_lotacao',
 'hierarquia_lotacao',
 'orgao',
 'orgao',
 'Ato_Exoneracao_Comissionado',
 'nome',
 'simbolo',
 'simbolo',
 'cargo_comissionado',
 'hierarquia_lotacao',
 'hierarquia_lotacao',
 'orgao',
 'orgao',
 'motivo',
 'simbolo',
 'orgao',
 'Ato_Exoneracao_Comissionado',
 'nome',
 'matricula',
 'cargo_comissionado',
 'hierarquia_lotacao',
 'Ato_Exoneracao_Comissionado',
 'nome',
 'simbolo',
 'simbolo',
 'cargo_comissionado',
 'orgao',
 'hierarquia_lotacao',
 'orgao',
 'orgao',
 'Ato_Exoneracao_Comissionado',
 'motivo',
 'nome',
 'simbolo',
 'cargo_comissionado',
 'hierarquia_lotacao',
 'motivo',
 'simbolo',
 'orgao',
 'hierarquia_lotacao',
 'Ato_Exoneracao_Comissionado',
 'cargo_efetivo',
 'nome',
 'nome',
 'matricula',
 'cargo_comissi

## Mapeando métricas de similaridade

Agora que já temos alguns procedimentos e estruturas auxiliares para medir similaridade entre os textos de anotações, vamos lá! A primeira coisa a se destacar é que temos alguns textos de anotações que foram anotados mais de 2 vezes, como mostra o mapa que fizemos mais cedo:

In [54]:
map_duplicata_final

{304: [15160],
 305: [1494],
 312: [9320, 9443, 9566, 9677],
 314: [15161],
 315: [1500],
 323: [15162],
 338: [9321, 9444, 9567, 9678],
 356: [1495],
 359: [12575],
 363: [9322, 9445, 9568, 9679, 15163],
 364: [9323, 9446, 9569, 9680],
 369: [12576],
 378: [12577],
 407: [9324, 9447, 9570, 9681],
 414: [15164],
 424: [15165],
 430: [1496],
 435: [9325, 9448, 9571, 9682],
 439: [12578],
 440: [1497],
 443: [15166],
 445: [9326, 9449, 9572, 9683],
 446: [9327, 9450, 9573, 9684],
 449: [12579],
 451: [1498],
 465: [12580],
 466: [1499],
 468: [9328, 9451, 9574, 9685],
 502: [1446],
 508: [1447],
 522: [9329, 9452, 9575, 9686],
 556: [9389, 9512, 9746, 9776],
 566: [1448],
 577: [9390, 9513, 9747, 9777, 15167],
 588: [9391, 9514, 9748, 9778],
 594: [1449],
 612: [15152],
 622: [15153],
 632: [9392, 9515, 9749, 9779],
 641: [9393, 9516, 9750, 9780],
 663: [15154],
 664: [15155],
 671: [9394, 9517, 9751, 9781],
 681: [1450],
 692: [9395, 9518, 9752, 9782],
 694: [12581],
 703: [15156],
 715

Nesse intuito, a estratégia adotada aqui foi criar combinações para cada uma das anotações feitas mais de uma vez. Para tanto, usamos o `itertools.combinations`. A ideia é calcular médias das combinações para cada métrica e salvar isso numa lista. Quando o elemento da vez for único, incluimos na lista 'el_unico'.

In [55]:
import itertools

In [312]:
cos_sim_lista = []
eucl_dist_lista = []
jacc_sim_lista = []

for offset in tqdm(offsets):
    if len(map_duplicata_final[offset]) > 1:
        qtd_comb = 0
        soma_cos = 0
        soma_eucl = 0
        soma_jc_char = 0
        # quero combinações dois a dois dos valores indexados por map_duplicata
        for combinacao in itertools.combinations(map_duplicata_final[offset], 2):
            #vector_1 = get_vector(data_dict[combinacao[0]][3])
            vector_1 = map_texto_vetores[combinacao[0]]
            #vector_2 = get_vector(data_dict[combinacao[1]][3])
            vector_2 = map_texto_vetores[combinacao[1]]
            cos = cos_sim(vector_1, vector_2)
            eucl = euclidean(vector_1, vector_2)
            jc = jaccard_char(data_dict[combinacao[0]][4], data_dict[combinacao[1]][4])
            soma_cos += cos
            soma_eucl += eucl
            soma_jc_char += jc
            qtd_comb += 1
        media_cos = soma_cos/qtd_comb
        media_eucl = soma_eucl/qtd_comb
        media_jc = soma_jc_char/qtd_comb
        cos_sim_lista.append(media_cos)
        eucl_dist_lista.append(media_eucl)
        jacc_sim_lista.append(media_jc)
    else:
        cos_sim_lista.append('el_unico')
        eucl_dist_lista.append('el_unico')
        jacc_sim_lista.append('el_unico')

  0%|          | 0/19492 [00:00<?, ?it/s]

Podemos ver que as listas possuem o mesmo número de linhas que o dataframe original.

In [313]:
len(cos_sim_lista), len(eucl_dist_lista), len(jacc_sim_lista)

(19492, 19492, 19492)

Para ficar mais bonitinho, vamos incluir isso no próprio dataframe:

In [314]:
df['sim_cos'] = cos_sim_lista
df['dist_eucl'] = eucl_dist_lista
df['sim_jacc'] = jacc_sim_lista

`sim_cos`, `dist_eucl`, `sim_jacc` foram as colunas adicionadas. 
- Para `sim_cos`, quanto mais próximo de 1, mais parecido. Se for = 1, é a mesma string;
- Para `dist_eucl`, quanto mais próximo de 0 melhor. Estamos falando de distância, então se a distância é 0, do ponto de vista vetorial é o mesmo elemento;
- Para `sim_jacc`, quanto mais próximo de 1, mais parecido. Se for = 1, é a mesma string.

In [315]:
df

,id_dodf,tipo_rel,id_rel,anotador_rel,tipo_ent,id_ent,anotador_ent,offset,length,texto,index,sim_cos,dist_eucl,sim_jacc
0,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,Ato_Exoneracao_Comissionado,1178,pedro_henrique,205008,326,EXONERAR SEBASTIAO FRANCISCO DE QUEIROZ do Car...,0,1.0,0.0,1.0
1,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,nome,1179,DODFMiner,205016,31,SEBASTIAO FRANCISCO DE QUEIROZ,1,1.0,0.0,1.0
2,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,simbolo,1181,DODFMiner,205078,4,DFA-,2,0.953665,1.126571,0.755102
3,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,simbolo,1180,pedro_henrique,205078,7,DFA-\n12,3,0.953665,1.126571,0.755102
4,17_133.28.6.2013,Ato_Exoneracao_Comissionado,R3,matheus_stauffer,cargo_comissionado,1182,DODFMiner,205089,9,Assessor,4,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19487,13_212.10.10.2013,Ato_Substituicao,R40,khalil_carsten,matricula_substituido,8685,vinicius_borges,644324,9,174.833-5,19487,el_unico,el_unico,el_unico
19488,13_212.10.10.2013,Ato_Substituicao,R40,khalil_carsten,data_inicial,8686,vinicius_borges,644348,2,09,19488,el_unico,el_unico,el_unico
19489,13_212.10.10.2013,Ato_Substituicao,R40,khalil_carsten,data_final,8687,vinicius_borges,644353,10,11/10/2013,19489,el_unico,el_unico,el_unico
19490,13_212.10.10.2013,Ato_Substituicao,R40,khalil_carsten,motivo,8688,vinicius_borges,644380,32,ferias regulamentares da titular,19490,el_unico,el_unico,el_unico


## Experimento de classificação

Usaremos as embeddings geradas para caracterização do texto.

In [ ]:
df_cls = df[['tipo_ent', 'texto', 'index']]
df_cls

,tipo_ent,texto,index
0,Ato_Exoneracao_Comissionado,EXONERAR SEBASTIAO FRANCISCO DE QUEIROZ do Car...,0
1,nome,SEBASTIAO FRANCISCO DE QUEIROZ,1
2,simbolo,DFA-,2
3,simbolo,DFA-\n12,3
4,cargo_comissionado,Assessor,4
...,...,...,...
19487,matricula_substituido,174.833-5,19487
19488,data_inicial,09,19488
19489,data_final,11/10/2013,19489
19490,motivo,ferias regulamentares da titular,19490


In [ ]:
df_cls.tipo_ent.value_counts()

nome                                  2022
orgao                                 1600
hierarquia_lotacao                    1512
cargo_comissionado                    1411
simbolo                               1358
                                      ... 
data_dodf_edital_normativo               3
data                                     3
fundamento_legal_abono_permanencia       3
tipo_edicao                              2
especialidade                            1
Name: tipo_ent, Length: 69, dtype: int64

In [ ]:
df_cls.loc[df_cls.tipo_ent == 'especialidade']

,tipo_ent,texto,index
9157,especialidade,AGENTE ADMINISTRATIVO,9157


In [ ]:
df_cls = df_cls[df_cls.tipo_ent != 'especialidade']
df_cls.tipo_ent.value_counts()

nome                                  2022
orgao                                 1600
hierarquia_lotacao                    1512
cargo_comissionado                    1411
simbolo                               1358
                                      ... 
data_dodf_resultado_final                3
matricula_siape                          3
data_dodf_edital_normativo               3
fundamento_legal_abono_permanencia       3
tipo_edicao                              2
Name: tipo_ent, Length: 68, dtype: int64

In [ ]:
map_texto_vetores_custom = map_texto_vetores
print(len(map_texto_vetores_custom))
map_texto_vetores_custom.pop(9157)
print(len(map_texto_vetores_custom))

19492
19491


In [ ]:
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split

In [ ]:
embedding_features = pd.DataFrame.from_dict(map_texto_vetores, orient='index')
embedding_features

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,105.940308,-17.377457,-35.207438,55.420986,-14.284359,-10.227432,22.412835,-13.948556,75.146801,-7.538433,53.286615,1.546869,-8.441187,-2.315337,-26.330482,61.497113,7.638495,-58.243785,34.650465,26.686081,23.001850,15.358394,25.949769,-15.425302,89.740496,-4.390033,50.755333,-31.037448,44.191691,11.401314,-7.831594,-13.446829,30.683326,25.576335,61.333663,71.733061,12.584169,17.305729,-13.641824,-0.511887,-53.968350,-19.611566,-11.634405,-16.460610,4.893647,-2.978188,-40.051534,22.592722,-23.578084,73.649694
1,-1.377036,1.049078,1.528914,-1.327946,-1.299674,0.181979,0.054252,-2.139108,5.113690,1.404221,-0.850023,-4.090120,0.854898,3.494136,0.694948,1.330963,-3.017321,-2.396978,0.401841,3.630240,2.666559,-2.243384,2.663682,1.364679,1.958922,1.097053,2.301511,-1.235244,3.145282,3.413187,0.599688,1.993799,0.616602,5.569513,2.617626,1.449382,1.150461,-1.281692,2.856875,-2.323158,0.098781,-1.553433,3.334604,0.270395,0.511851,2.453013,3.304414,1.335412,0.797231,5.529418
2,0.473798,0.410046,-0.033130,-0.229452,0.237273,0.342567,-0.383352,0.183555,0.338811,0.084414,0.107889,-0.096369,-0.286557,0.072725,0.020530,0.158989,-0.280927,-0.353242,-0.340550,0.462992,0.668817,-0.281560,0.264404,0.196495,0.542685,-0.100404,0.252819,-0.336500,-0.034371,0.263844,0.211052,0.103135,0.128815,0.259697,0.376384,0.055181,0.228828,-0.309801,-0.017903,0.130430,-0.138217,0.108359,-0.083940,-0.066280,-0.093860,0.231255,0.150242,0.596081,-0.233623,0.577339
3,0.794462,0.324704,-0.512305,-0.367634,0.244048,0.616679,-0.460489,0.698799,0.723257,-0.396285,0.213642,-0.262688,-0.285342,-0.226539,0.027318,0.105335,-0.570808,-0.861657,-0.485725,0.823502,1.079982,-0.326570,0.081728,0.465264,0.959549,-0.356378,0.011708,-0.516071,-0.256120,0.567400,0.503683,0.127551,0.477199,0.484889,0.728395,0.191996,0.418474,-0.342556,-0.305466,-0.097186,-0.231305,0.059904,-0.169201,-0.097291,-0.142533,0.418954,0.352128,0.856198,-0.420844,1.036525
4,3.225576,1.347287,-0.015522,0.700949,-0.384434,0.907945,-0.520805,-1.268238,1.826199,0.801131,1.270496,0.523419,-0.990024,0.729762,-0.554712,1.851162,-0.338523,-0.742688,0.153148,0.748993,2.397966,-1.177113,1.587106,-0.434090,2.517820,0.274871,1.501868,-0.660809,1.149783,0.419191,0.040668,-0.048179,0.811869,0.471353,0.576609,1.330303,0.459456,-0.524347,-0.381482,0.881522,-0.237228,0.398150,-0.565958,-0.823413,-0.042902,0.776817,-0.794645,1.942039,0.181388,1.864788
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19487,0.008901,0.006065,-0.013996,-0.008814,0.005923,0.009606,-0.006038,0.003689,-0.015493,-0.023631,-0.017037,0.018261,-0.007507,-0.010648,0.007346,-0.015383,0.024268,0.000112,0.001667,0.002142,-0.005034,-0.001459,-0.017926,0.012574,-0.003464,-0.006722,-0.021628,-0.009439,-0.013418,-0.004310,-0.004039,-0.012382,0.010286,-0.000900,-0.004823,-0.023756,-0.000894,-0.004445,-0.013704,0.012907,0.010131,-0.000527,0.008068,0.016021,-0.028338,-0.003657,-0.001172,-0.016295,0.003857,-0.016235
19488,0.214760,-0.164183,-0.520343,-0.222834,-0.051970,0.361736,0.031869,0.481082,0.422028,-0.611084,0.082006,-0.275258,-0.021185,-0.480083,-0.052930,-0.260438,-0.390497,-0.508089,-0.084227,0.442701,0.504461,0.019605,-0.354961,0.186929,0.260551,-0.358840,-0.304915,-0.103570,-0.427674,0.377308,0.390816,0.110821,0.273312,0.107104,0.318873,0.235827,0.219256,-0.002035,-0.470209,-0.135474,-0.072892,-0.034376,-0.118248,-0.154818,0.094056,0.238676,0.205652,0.245626,-0.180358,0.395369
19489,0.006193,-0.005127,-0.016287,-0.005761,-0.033202,0.013102,0.009968,0.018462,0.081478,-0.034526,-0.010212,-0.078791,0.006977,-0.008824,0.001584,0.007951,-0.062275,-0.070984,0.020987,0.063739,0.075305,-0.006560,-0.014194,0.018034,0.041650,-0.008867,0.013551,-0.020698,0.005975,0.0727

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_cls['texto'], df_cls['tipo_ent'], test_size=0.25, random_state=14, stratify=df_cls['tipo_ent'])

In [ ]:
emb_train, emb_test = train_test_split(embedding_features, test_size=0.25, random_state=14, stratify=df_cls['tipo_ent'])

print(emb_train.shape, emb_test.shape)

(14618, 50) (4873, 50)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

# fit_transform no treino
X_csr_train = vectorizer.fit_transform(X_train)
print(X_csr_train.shape)

# apenas transform no teste
X_csr_test = vectorizer.transform(X_test)
print(X_csr_test.shape)

(14618, 4543)
(4873, 4543)


In [ ]:
svm_model = LinearSVC(max_iter=1000, random_state=14, verbose=True)
svm_model.fit(X_csr_train, y_train)
svm_prediction_tfidf = svm_model.predict(X_csr_test)

[LibLinear]

In [ ]:
svm_w2v = LinearSVC(max_iter=1000, random_state=14, verbose=True)
svm_w2v.fit(emb_train, y_train)
svm_pred_w2v = svm_w2v.predict(emb_test)

[LibLinear]

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [ ]:
results = pd.DataFrame(columns = ['Precision', 'Recall', 'F1 score', 'support']
          )
results.loc['tfidf + svm'] = precision_recall_fscore_support(
          y_test, 
          svm_prediction_tfidf, 
          average = 'weighted'
          )
results.loc['w2v + svm'] = precision_recall_fscore_support(
          y_test, 
          svm_pred_w2v, 
          average = 'weighted'
          )

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
results

,Precision,Recall,F1 score,support
tfidf + svm,0.800548,0.820029,0.801241,NaN
w2v + svm,0.691884,0.708393,0.675069,NaN
